In [15]:
# coding=utf-8
# https://www.akshare.xyz/demo.html#id1
from datetime import datetime

import backtrader as bt  # 升级到最新版
import matplotlib.pyplot as plt  # 由于 Backtrader 的问题，此处要求 pip install matplotlib==3.2.2
import akshare as ak  # 升级到最新版
import pandas as pd

plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

In [16]:
# 利用 AKShare 获取股票的后复权数据，这里只获取前 6 列
stock_hfq_df = ak.stock_zh_a_hist(symbol="000001", adjust="hfq").iloc[:, :6]
# 处理字段命名，以符合 Backtrader 的要求
stock_hfq_df.columns = [
    'date',
    'open',
    'close',
    'high',
    'low',
    'volume',
]
stock_hfq_df

,date,open,close,high,low,volume
0,1991-04-03,49.00,49.00,49.00,49.00,1
1,1991-04-04,48.76,48.76,48.76,48.76,3
2,1991-04-05,48.52,48.52,48.52,48.52,2
3,1991-04-06,48.28,48.28,48.28,48.28,7
4,1991-04-08,48.04,48.04,48.04,48.04,2
...,...,...,...,...,...,...
7327,2021-12-15,3014.70,3021.21,3039.08,2996.83,1294627
7328,2021-12-16,3021.21,3048.84,3050.46,3006.58,819480
7329,2021-12-17,3045.58,3024.46,3053.71,3024.46,602370
7330,2021-12-20,3022.83,3016.33,3037.46,3004.95,746435


In [17]:
# 保存数据
stock_hfq_df.to_csv('./data/stock_hfq_df.txt')
type(stock_hfq_df)

pandas.core.frame.DataFrame

In [18]:


# 把 date 作为日期索引，以符合 Backtrader 的要求
stock_hfq_df.index = pd.to_datetime(stock_hfq_df['date'])

In [19]:
class MyStrategy(bt.Strategy):
    """
    主策略程序
    """
    params = (("ma5", 5), ("ma60", 60),)  # 全局设定交易策略的参数

    def __init__(self):
        """
        初始化函数
        """
        self.data_close = self.datas[0].close  # 指定价格序列
        # 初始化交易指令、买卖价格和手续费
        # 用于保存订单
        self.order = None
        # 订单价格
        self.buy_price = None
        # 订单佣金
        self.buy_comm = None
        # 添加移动均线指标
        self.ma5 = bt.indicators.MovingAverageSimple(self.datas[0], period=self.params.ma5)
        self.ma60 = bt.indicators.MovingAverageSimple(self.datas[0], period=self.params.ma60)

    # 策略逻辑实现
    def next(self):
        """
        执行逻辑
        """
        if self.order:  # 检查是否有指令等待执行,
            return
        # 检查是否持仓
        if not self.position:  # 没有持仓
            # 当今天的5日均线大于60日均线并且昨天的5日均线小于60日均线，则进入市场（买）
            if self.ma5[0] > self.ma60[0] and self.ma5[-1] < self.ma60[-1]:
                self.order = self.buy(size=100)  # 执行买入
        else:
            if self.ma5[0] < self.ma60[0] and self.ma5[-1] > self.ma60[-1]:
                self.order = self.sell(size=100)  # 执行卖出


In [20]:
cerebro = bt.Cerebro()  # 初始化回测系统
start_date = datetime(1991, 4, 3)  # 回测开始时间
end_date = datetime(2020, 6, 16)  # 回测结束时间
data = bt.feeds.PandasData(dataname=stock_hfq_df, fromdate=start_date, todate=end_date)  # 加载数据
cerebro.adddata(data)  # 将数据传入回测系统
cerebro.addstrategy(MyStrategy)  # 将交易策略加载到回测系统中
start_cash = 1000000
cerebro.broker.setcash(start_cash)  # 设置初始资本为 100000
cerebro.broker.setcommission(commission=0.002)  # 设置交易手续费为 0.2%
cerebro.run()  # 运行回测系统

port_value = cerebro.broker.getvalue()  # 获取回测结束后的总资金
pnl = port_value - start_cash  # 盈亏统计

print(f"初始资金: {start_cash}\n回测期间：{start_date.strftime('%Y%m%d')}:{end_date.strftime('%Y%m%d')}")
print(f"总资金: {round(port_value, 2)}")
print(f"净收益: {round(pnl, 2)}")

cerebro.plot(style='candlestick')  # 画图

初始资金: 1000000
回测期间：19910403:20200616
总资金: 1218685.48
净收益: 218685.48


[[<Figure size 540x360 with 4 Axes>]]